In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from time import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import seaborn as sns
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
import itertools
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.regularizers import l1_l2
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from scipy.stats import norm
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller,kpss
from arch.unitroot import PhillipsPerron

import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Function to convert datetime to date only in DataFrame

In [3]:
# S would be the DataFrame that we want to convert
def date_time(S):
    S.index = S.index.date
    S.index = pd.to_datetime(S.index)
    return S

# Function to convert quarterly data to monthly using Forwardfill

In [7]:
'''
    S = Dataframe to convert
    n = Dividing factor (usually 1 if data is already in percentage form) 
''' 

def quarterly_to_monthly_forwardfill(S,n):
    monthly = pd.DataFrame(S.dropna())
    monthly.index = monthly.index.shift(3,freq='MS')
    monthly = (monthly.asfreq('MS',method='ffill')/n).resample('M').max()
    return monthly

# Function to create correlation heatmap for leading and lagging indicators

In [6]:
'''
    data1 = Will contain dataframe
    numshifts = Numbers of lead or lags that we want for our data
    zero_or_one = 1 is for leading indicators and 0 for lagging indicators
''' 

def leadlagheatmap(data1, numshifts, zero_or_one):
    plt.figure(figsize=(15, 13))
    if zero_or_one == 0:
        target = data1.iloc[:,-1].shift(numshifts)
        vars = data1.iloc[0:-numshifts,0:-1]
        comblagged = pd.concat([vars, target], axis=1)
        comblagged.dropna
    else:
        target = data1.iloc[0:-numshifts,-1]
        vars = data1.iloc[:,0:-1].shift(numshifts)
        comblagged = pd.concat([vars, target], axis=1)
        comblagged.dropna
    return sns.heatmap(comblagged.corr(),annot=True)

# Function to create training and testing data for time series analysis

In [8]:
'''
    dep_data = Target data
    indep_data = Predictors
    shufflestate = Usually False if we want to maintain temporal order of time series data
    testsize = Usually between 20-30%
'''
def traintestsplit(dep_data,indep_data, shufflestate, testsize):
    x_train, x_test, y_train, y_test = train_test_split(dep_data, indep_data, shuffle=shufflestate, test_size=testsize)
    return x_train, x_test, y_train, y_test

# Function to check for stationarity using KPSS, ADF and Philips Perron test

In [9]:
'''
    varname = These are the column names of our dataframe
    vardata = these are the datapoints of our dataframe
    
    The output would tell us whether each and every variable is stationary or not based on all 3 tests of stationarity
'''
def int_stat_test(varname, vardata):
    # ADF Test
    adfcnt = 0
    adf = adfuller(vardata)
    if adf[1] < 0.05:
        adfcnt = 1
    # KPSS Test
    kpcnt = 0
    kpstest = kpss(vardata)
    if kpstest[1] > 0.05:
        kpcnt = 1
    # PP Test
    ppcnt = 0
    pptest = PhillipsPerron(vardata)
    if pptest.pvalue< 0.05:
        ppcnt = 1
    if adfcnt + kpcnt + ppcnt >= 2:
        print(varname, 'is stationary')
    else:
        print(varname, 'is not stationary')

# Function to plot components of our time series model

In [11]:
'''
    S = Dataframe to input
    model_type = Usually 'additive'. 'multiplicative' models involve a logarithmic component which does not work for 
    negative returns
'''
def seasonalty_check(S,model_type):
    data_columns = S.columns
    for column in data_columns:
        seasonal_decomposition = sm.tsa.seasonal_decompose(S[column], model=model_type)

        # Plot the original time series, trend, seasonal, and residual components
        fig, ax = plt.subplots(4, 1, figsize=(10, 8))
        ax[0].plot(S.index, S[column], label='Original')
        ax[0].legend(loc='best')
        ax[1].plot(S.index, seasonal_decomposition.trend, label='Trend')
        ax[1].legend(loc='best')
        ax[2].plot(S.index, seasonal_decomposition.seasonal, label='Seasonal')
        ax[2].legend(loc='best')
        ax[3].plot(S.index, seasonal_decomposition.resid, label='Residual')
        ax[3].legend(loc='best')

        # Set the title of the plot as the column name
        ax[0].set_title(column)

    # Show the plots
    return plt.tight_layout()
    return plt.show()

# Function to create PACF and ACF Plots

In [12]:
def PACF_ACF_plot(S):
    for column in range(len(S.columns)):
        # Plot PACF
        fig, ax = plt.subplots(figsize=(10, 4))
        plot_pacf(S.iloc[:,column], lags=50, ax=ax)
        ax.set_title('Partial Autocorrelation Function (PACF)')
        plt.title(S.columns[column] + ' PACF')
        plt.show()

        # Plot ACF
        fig, ax = plt.subplots(figsize=(10, 4))
        plot_acf(S.iloc[:,column], lags=50, ax=ax)
        ax.set_title('Autocorrelation Function (ACF)')
        plt.title(S.columns[column] + ' ACF')
        plt.show()

# Function to decompose our time series data

In [13]:
def seasonality_trend_residual(S, model_type):
    ccardata_seasonal = pd.DataFrame(index=S.index, columns=S.columns)
    ccardata_trend = pd.DataFrame(index=S.index, columns=S.columns)
    ccardata_residual = pd.DataFrame(index=S.index, columns=S.columns)
    ccardata_SA = pd.DataFrame(index=S.index, columns=S.columns)
    
    for column in S.columns:
        seasonal_decomposition = sm.tsa.seasonal_decompose(S[column], model=model_type)
        deseasonalized = S[column] - seasonal_decomposition.seasonal

        
        ccardata_SA.loc[:, column] = deseasonalized
        ccardata_seasonal.loc[:, column] = seasonal_decomposition.seasonal
        ccardata_trend.loc[:, column] = seasonal_decomposition.trend
        ccardata_residual.loc[:, column] = seasonal_decomposition.resid
        

        
    return ccardata_seasonal, ccardata_trend, ccardata_residual,ccardata_SA

# ARIMA Model Functions

## Function to check best pdq order for ARIMA based on RMSE

In [14]:
'''
    p_range, d_range, q_range = range of p,d,q values that we want to test
    train_y = training set of our target variable
    test_y = testing set of our target variable
'''
def ARIMA_pdq_order(p_range, d_range, q_range, train_y, test_y):
    p_values = range(0, p_range)
    d_values = range(0, d_range)
    q_values = range(0, q_range)
    
    # Initialize variables
    best_rmse = float('inf')
    best_order = None
    
    # Generate all combinations of p, d, q values
    order_combinations = list(itertools.product(p_values, d_values, q_values))
    
    for order in order_combinations:
        try:
            # Create and fit the ARIMA model
            model = sm.tsa.ARIMA(train_y, order=order)
            results = model.fit()
            forecast_ARIMA = results.forecast(steps=len(test_y))
            rmse = np.sqrt(mean_squared_error(test_y, forecast_ARIMA))
            
            # Check if current RMSE is better than the previous best
            if rmse < best_rmse:
                best_rmse = rmse
                best_order = order
        except:
            continue
                    
    # Print the best order values and RMSE
    return "Best Order:", best_order, "Best RMSE:", best_rmse

## Function to give a table of statistical values for comparison of ARIMA Model

In [15]:
'''
    order = Best pdq order we got from above code
'''
def ARIMA_order_table(order, train_y, test_y):

    # Create and fit the ARIMA model
    model = sm.tsa.ARIMA(train_y, order=order)
    results = model.fit()
    forecast_ARIMA = results.forecast(steps=len(test_y))
    rmse = np.sqrt(mean_squared_error(test_y, forecast_ARIMA))
    mape = np.mean(np.abs((test_y - forecast_ARIMA) / test_y)) * 100
    mae = mean_absolute_error(test_y, forecast_ARIMA)
    r_squared = r2_score(test_y, forecast_ARIMA)
    aic = results.aic
    bic = results.bic

    # Append the results to the table
    results_df = pd.DataFrame([[order, rmse, mape, mae, r_squared,aic, bic]],
                              columns=["Order", "RMSE", "MAPE", "MAE", "R-Squared","AIC", "BIC"])

    # Print the results in tabular form
    return (results_df)


## Function to find in-sample and out-sample RMSE and predictions of ARIMA

In [16]:
def ARIMA_model_in(train_y,test_y,order):
    model_ARIMA = sm.tsa.ARIMA(train_y, order=order)
    results_ARIMA = model_ARIMA.fit()
    forecast_ARIMA = results_ARIMA.forecast(steps=len(test_y))
    predict = results_ARIMA.predict(start = '1997-01-31', end ='2017-12-31')
    rmse = np.sqrt(mean_squared_error(predict,train_y))
    return rmse,predict

def ARIMA_model_out(train_y,test_y,order):
    model_ARIMA = sm.tsa.ARIMA(train_y, order=order)
    results_ARIMA = model_ARIMA.fit()
    forecast_ARIMA = results_ARIMA.forecast(steps=len(test_y))
    rmse = np.sqrt(mean_squared_error(forecast_ARIMA,test_y))
    summary = results_ARIMA.summary()
    return rmse,forecast_ARIMA,summary

# ARIMAX Model Functions

## Function to check best pdq order for ARIMAX based on RMSE

In [17]:
'''
    Similar to ARIMA model. We have just included an exogenous variable here
'''

def ARIMAX_pdq_order(p_range, d_range, q_range, train_x, test_x, train_y, test_y):
    p_values = range(0, p_range)
    d_values = range(0, d_range)
    q_values = range(0, q_range)

    # Initialize variables
    best_rmse = float('inf')
    best_order = None

    # Generate all combinations of p, d, q values
    order_combinations = list(itertools.product(p_values, d_values, q_values))

    for order in order_combinations:
        try:
            # Create and fit the ARIMAX model
            model = sm.tsa.ARIMA(train_y, order=order, exog=train_x)
            results = model.fit()
            forecast_ARIMAX = results.forecast(steps=len(test_y), exog=test_x)
            rmse = np.sqrt(mean_squared_error(test_y, forecast_ARIMAX))

            # Check if current RMSE is better than the previous best
            if rmse < best_rmse:
                best_rmse = rmse
                best_order = order

        except:
            continue

    # Print the best order values and RMSE
    return "Best Order:", best_order, "Best RMSE:", best_rmse

## Function to give a table of statistical values for comparison of ARIMAX Model

In [18]:
def ARIMAX_order_table(train_x, test_x, train_y, test_y, order):
    # Create and fit the ARIMAX model
    model = sm.tsa.ARIMA(train_y, exog=train_x, order=order)
    results = model.fit()
    forecast_ARIMAX = results.forecast(steps=len(test_y), exog=test_x)
    rmse = np.sqrt(mean_squared_error(test_y, forecast_ARIMAX))
    mape = np.mean(np.abs((test_y - forecast_ARIMAX) / test_y)) * 100
    mae = mean_absolute_error(test_y, forecast_ARIMAX)
    r_squared = r2_score(test_y, forecast_ARIMAX)
    aic = results.aic
    bic = results.bic

    # Create a DataFrame to store the results
    results_df = pd.DataFrame([[order, rmse, mape, mae, r_squared,aic, bic]],
                              columns=["Order", "RMSE", "MAPE", "MAE", "R-Squared","AIC", "BIC"])

    # Print the results in tabular form
    return results_df

## Function to find in-sample and out-sample RMSE and predictions of ARIMA

In [19]:
def ARIMAX_model_in(train_x,test_x,train_y,test_y,order):
    model_ARIMAX = sm.tsa.ARIMA(train_y, order=order, exog=train_x)
    results_ARIMAX = model_ARIMAX.fit()
    forecast_ARIMAX = results_ARIMAX.forecast(steps=len(test_y), exog=test_x)
    predict = results_ARIMAX.predict(start = '1997-01-31', end ='2017-12-31')
    rmse = np.sqrt(mean_squared_error(predict,train_y))
    return rmse,predict

def ARIMAX_model_out(train_x,test_x,train_y,test_y,order):
    model_ARIMAX = sm.tsa.ARIMA(train_y, order=order, exog=train_x)
    results_ARIMAX = model_ARIMAX.fit()
    forecast_ARIMAX = results_ARIMAX.forecast(steps=len(test_y), exog=test_x)
    rmse = np.sqrt(mean_squared_error(forecast_ARIMAX,test_y))
    summary = results_ARIMAX.summary()
    return rmse,forecast_ARIMAX,summary

# ARIMAX-GARCH Model Functions

## Function to check best pdq order for ARIMAX-GARCH based on RMSE

In [20]:
'''
    We have to input our training and testing set of our Independent and Dependent variables
'''
def find_best_arimax_garch_order(trainx_combined_SA, testx_combined_SA, trainy_combined_SA,testy_combined_SA):
    p_values = range(1, 4)
    d_values = range(0, 4)
    q_values = range(1, 4)
    pdq = list(itertools.product(p_values, d_values, q_values))

    P_values = range(1, 4)
    Q_values = range(1, 4)
    pq = list(itertools.product(P_values, Q_values))

    best_rmse = float('inf')
    best_arima_order = None
    best_garch_order = None

    for arima_order in pdq:
        for garch_order in pq:
            # Create and fit the ARIMAX model
            model = sm.tsa.ARIMA(trainy_combined_SA, order=arima_order, exog=trainx_combined_SA)
            arima_results = model.fit()
            arima_residuals = arima_results.resid
            arima_pred = arima_results.forecast(steps=len(testy_combined_SA), exog=testx_combined_SA)

            garch_model = arch.arch_model(arima_residuals, vol='Garch', p=garch_order[0], q=garch_order[1])
            results = garch_model.fit()
            garch_pred = results.forecast(horizon=len(testy_combined_SA))
            garch_var = np.sqrt(garch_pred.variance.iloc[-1, :])
            garch_var.index = testy_combined_SA.index
            garch_mean = garch_var.mean()
            total = garch_mean + arima_pred
            rmse = np.sqrt(mean_squared_error(testy_combined_SA, total))

            if rmse < best_rmse:
                best_rmse = rmse
                best_arima_order = arima_order
                best_garch_order = garch_order

    return best_arima_order, best_garch_order

## Function to give a table of statistical values for comparison of ARIMAX Model

In [21]:
def arimax_garch_analysis(trainx_combined_SA, testx_combined_SA, trainy_combined_SA,testy_combined_SA):
    # Fit ARIMAX model
    model_arimax = sm.tsa.ARIMA(trainy_combined_SA, order=(2,0,2), exog=trainx_combined_SA)
    arimax_results = model_arimax.fit()
    arimax_residuals = arimax_results.resid
    
    # Perform GARCH modeling
    garch_model = arch.arch_model(arimax_residuals, vol='Garch', p=3, q=3)
    garch_results = garch_model.fit()
    
    # Generate ARIMAX-GARCH predictions
    arimax_pred = arimax_results.forecast(steps=len(testy_combined_SA), exog=testx_combined_SA)
    garch_pred = garch_results.forecast(horizon=len(testy_combined_SA))
    a = np.sqrt(garch_pred.variance.iloc[-1, :])
    a.index = testy_combined_SA.index
    a_mean = a.mean()
    total = arimax_pred + a_mean
    
    # Calculate performance metrics
    rmse = np.sqrt(mean_squared_error(testy_combined_SA, total))
    mape = np.mean(np.abs((testy_combined_SA - total) / testy_combined_SA)) * 100
    mae = mean_absolute_error(testy_combined_SA, total)
    r_squared = r2_score(testy_combined_SA, total)
    aic = arimax_results.aic + garch_results.aic
    bic = arimax_results.bic + garch_results.bic
    
    # Create output table
    output_table = pd.DataFrame({
        'ARIMAX-GARCH Order': [f'ARIMA{(2,0,2)}-GARCH({(3)}, {(3)})'],
        'RMSE': [rmse],
        'MAPE': [mape],
        'MAE': [mae],
        'R-squared': [r_squared],
        'AIC': [aic],
        'BIC': [bic]
    })
    
    return output_table

# SARIMAX Model Functions

## Function to check best pdq order for SARIMAX based on RMSE

In [22]:
def SARIMAX_order(X_train, X_test, Y_train,Y_test,s):
    p_values = range(1, 3)
    d_values = range(0, 3)
    q_values = range(1, 3)
    pdq = list(itertools.product(p_values, d_values, q_values))

    P_values = range(1, 3)
    D_values = range(0, 3)
    Q_values = range(1, 3)
    S_values = s
    pdqs = list(itertools.product(P_values, D_values, Q_values, S_values))

    best_rmse = float('inf')
    best_order = None
    best_seasonal_order = None

    for i in range(len(pdq)):
        for j in range(len(pdqs)):
            # Create and fit the SARIMAX model
            model = sm.tsa.SARIMAX(Y_train, order=pdq[i], seasonal_order=pdqs[j], exog=X_train)
            results = model.fit()
            forecast_SARIMAX = results.forecast(steps=len(Y_test), exog=X_test)
            rmse = np.sqrt(mean_squared_error(Y_test, forecast_SARIMAX))
            if rmse < best_rmse:
                best_rmse = rmse
                best_order = pdq[i]
                best_seasonal_order = pdqs[j]

    print(f"Order: {best_order}, Seasonal Order: {best_seasonal_order}, RMSE: {best_rmse}")

## Function to give a table of statistical values for comparison of SARIMAX Model

In [23]:
def SARIMAX_order_table(train_x, test_x, train_y, test_y, order, seasonal_order):
    # Create and fit the SARIMAX model
    model = sm.tsa.SARIMAX(train_y, exog=train_x, order=order, seasonal_order=seasonal_order)
    results = model.fit()
    forecast_SARIMAX = results.forecast(steps=len(test_y), exog=test_x)
    rmse = np.sqrt(mean_squared_error(test_y, forecast_SARIMAX))
    mape = np.mean(np.abs((test_y - forecast_SARIMAX) / test_y)) * 100
    mae = mean_absolute_error(test_y, forecast_SARIMAX)
    r_squared = r2_score(test_y, forecast_SARIMAX)
    aic = results.aic
    bic = results.bic

    # Create a DataFrame to store the results
    results_df = pd.DataFrame([[order, seasonal_order, rmse, mape, mae, r_squared, aic, bic]],
                              columns=["Order", "Seasonal Order", "RMSE", "MAPE", "MAE", "R-squared", "AIC", "BIC"])

    # Print the results in tabular form
    return results_df

## Function to find in-sample and out-sample RMSE and predictions of SARIMAX

In [24]:
def SARIMAX_model_in(train_x, test_x, train_y, test_y, order, seasonal_order):
    model_SARIMAX = sm.tsa.SARIMAX(train_y, exog=train_x, order=order, seasonal_order=seasonal_order)
    results_SARIMAX = model_SARIMAX.fit()
    forecast_SARIMAX = results_SARIMAX.forecast(steps=len(test_y), exog=test_x)
    predict = results_SARIMAX.predict(start='1997-01-31', end='2017-12-31')
    rmse = np.sqrt(mean_squared_error(predict, train_y))
    r_squared = r2_score(train_y, predict)
    return rmse, predict,r_squared

def SARIMAX_model_out(train_x, test_x, train_y, test_y, order, seasonal_order):
    model_SARIMAX = sm.tsa.SARIMAX(train_y, exog=train_x, order=order, seasonal_order=seasonal_order)
    results_SARIMAX = model_SARIMAX.fit()
    forecast_SARIMAX = results_SARIMAX.forecast(steps=len(test_y), exog=test_x)
    rmse = np.sqrt(mean_squared_error(forecast_SARIMAX, test_y))
    r_squared = r2_score(test_y, forecast_SARIMAX)
    summary_table = results_SARIMAX.summary()
    return rmse, forecast_SARIMAX,r_squared,summary_table